# GUESS MY WEIGHT 

![guess_your_weight.gif](images/guess_your_weight.gif)

## Table of Contents TOC
[Overview](#overview)<br />
[Data Understanding](#data-understanding)<br />
[Data Preparation](#data-preparation)<br />
[Modeling](#modeling)<br />
[Evaluation](#evaluation)<br />
[Github Repository and Resources](#github-repository-and-resources)<br />


## Overview
Health and Wellness is a big business. Specifically, weight loss. We’re all trying because it’s very, very hard. I recently went on my own weight loss journey, losing about 50 lbs in roughly 18 months. Weighing myself every morning, I agonized over every tenth of a lb, recording it in an app on my phone. I realized that losing big chunks of weights starts with small, incremental progress on the scale. But I didn’t stop there. As a data nerd I thought, “let’s record every meal.” So I did that too. I wondered… given all this data I have, could I predict my weight? My watch and phone captures my exercise, sleep, eating, and so much more. There must be trends here. At a minimum, I should be able to predict whether my weight will go up or down from the previous day. So let’s do it.<br />
[return to TOC](#table-of-contents-TOC)

## Data Understanding
I have much (and probably too much) of this data in my iphone and Apple Watch. It contains the weight information, workouts, heart rate, meals - broken down into subcategories (proteins, fats, etc). Most importantly is the weight. That will be the feature that I primarily use for classification.  

Because it’s my data, there’s more clarity about data entry methods. This is more subjective, than a controlled experiment with many participants. I know what data I was diligent about collecting so I should be able to scrub it appropriately. For instance, I didn’t record my fluids consistently - water, tea, coffee. Water consumption is a big part of this so I’ll have to be clear about the gaps in the data.<br />
[return to TOC](#table-of-contents-TOC)


## Data Preparation
The data is stored on a csv file in a kaggle repository.

in an xml file on my phone. After downloading it into python notebook and digging a little, there are roughly 180 rows of weight entries (approximately 6 months) but it’s not clear how many gaps there are. All of the data is stored as an entry, with time stamps and usually some numeric form. Whether it’s heart rate, weight, caloric info, it’s one numeric entry with an associated units. We’re primarily dealing with ints and floats, all numeric, and we’ll be using daily totals/averages. Because we only have one weigh-in per day, we’re only going to use daily values of other data. So… we know we have approximately 100-180 rows. I can’t say at the moment how many columns, because this will be based on what happens in pre-processing. Which brings me to../.

There are two major challenges with the pre-processing. The first deals with the privacy of my personal health data. How do I balance reproducibility requirements with privacy concerns? I need to make the dataset publicly available, including all of my pre-processing steps, but I also want to make sure no one can link it back to me, Andrew Q. Bennett (my real middle name doesn’t start with Q… gotcha!!!!). And the initial dataset is large, maybe 40 MB. The approach we’ll use is to perform some pre-processing locally, and then upload to the kaggle site when it’s ready for public consumption. In my jupyter notebook, I will comment out some of this code so that we can see the work, but it won’t affect the code when we press “run”.

The second is dealing with correlation efforts. For instance, we know that all data related to working out is going to be correlated with eachother. The steps, average heart rate, workout calories, etc will all be correlated to whether I went for a jog that day. Making decisions about which data to use will be a challenge, even with some baseline domain knowledge. There is a treasure trove that may have nothing (or very little) to do with weight loss, like Vitamin A intake. PCA Analysis will be critical without losing some data. I know about health…but I’m no expert. Maybe Vitamin A intake can help/hurt weight loss.

The many visualization efforts will come from making sure the weight data is presented cleanly. A nice, regression line showing weight trends over different periods will be very helpful.<br />
[return to TOC](#table-of-contents-TOC)

In [112]:
import pandas as pd

In [113]:
df = pd.read_csv('pre_kaggle/weight_data.csv')
df

,Unnamed: 0,type,unit,creationDate,startDate,endDate,value
0,793917,HKCategoryTypeIdentifierSleepAnalysis,HKCategoryValueSleepAnalysisInBed,2023-07-24 05:30:00,2023-07-23 21:52:17,2023-07-23 21:53:47,1.0000
1,793918,HKCategoryTypeIdentifierSleepAnalysis,HKCategoryValueSleepAnalysisInBed,2023-07-24 05:30:00,2023-07-23 22:13:46,2023-07-23 22:13:59,1.0000
2,793919,HKCategoryTypeIdentifierSleepAnalysis,HKCategoryValueSleepAnalysisInBed,2023-07-24 05:30:00,2023-07-23 22:14:56,2023-07-23 22:58:00,1.0000
3,793920,HKCategoryTypeIdentifierSleepAnalysis,HKCategoryValueSleepAnalysisAsleepCore,2023-07-24 07:05:22,2023-07-23 23:12:48,2023-07-23 23:40:48,1.0000
4,793921,HKCategoryTypeIdentifierSleepAnalysis,HKCategoryValueSleepAnalysisAsleepDeep,2023-07-24 07:05:22,2023-07-23 23:40:48,2023-07-24 00:06:18,1.0000
...,...,...,...,...,...,...,...
856980,856980,HKQuantityTypeIdentifierHeartRateVariabilitySDNN,ms,2024-03-06 01:01:10,2024-03-06 01:00:08,2024-03-06 01:01:07,44.3289
856981,856981,HKQuantityTypeIdentifierHeartRateVariabilitySDNN,ms,2024-03-06 03:01:11,2024-03-06 03:00:09,2024-03-06 03:01:08,54.3759
856982,856982,HKQuantityTypeIdentifierHeartRateVariabilitySDNN,ms,2024-03-06 05:01:11,2024-03-06 05:00:09,2024-03-06 05:01:08,76.2300
856983,856983,HKQuantityTypeIdentifierHeartRateVariabilitySDNN,ms,2024-03-06 07:01:22,2024-03-06 07:00:20,2024-03-06 07:01:20,45.6944


### EDA - Prescrubbing
Some EDA was previously performed to get the dataset into Kaggle. Now, of course, we have to get our data oriented for our analysis. We know we want our daily weigh-in data to be our target feature. We also know we want one daily value for each variable feature. For instance, there's multiple data on sleep, but perhaps we only care about the total hours that we're slept. First, let's strip the data type descriptions to something readable.

I want to create a table with weight as the target with only one value for each day.

1. Separate sleep and non-sleep - create a check to separate daily values with non-daily values.
    Sample, create table with daily values.
2. Create a 
 

In [114]:
df.sort_values('startDate')

,Unnamed: 0,type,unit,creationDate,startDate,endDate,value
267737,254050,HKQuantityTypeIdentifierBasalEnergyBurned,Cal,2023-07-23 08:05:38,2023-07-23 00:43:05,2023-07-23 01:43:05,76.125
267738,254051,HKQuantityTypeIdentifierBasalEnergyBurned,Cal,2023-07-23 08:05:38,2023-07-23 01:43:05,2023-07-23 02:43:05,76.125
267739,254052,HKQuantityTypeIdentifierBasalEnergyBurned,Cal,2023-07-23 08:05:38,2023-07-23 02:43:05,2023-07-23 03:43:05,76.125
267740,254053,HKQuantityTypeIdentifierBasalEnergyBurned,Cal,2023-07-23 08:05:38,2023-07-23 03:43:05,2023-07-23 04:43:05,76.125
267741,254054,HKQuantityTypeIdentifierBasalEnergyBurned,Cal,2023-07-23 08:05:38,2023-07-23 04:43:05,2023-07-23 05:43:05,76.125
...,...,...,...,...,...,...,...
807601,793914,HKQuantityTypeIdentifierPhysicalEffort,kcal/hr·kg,2024-03-06 09:52:33,2024-03-06 09:50:45,2024-03-06 09:50:53,4.800
807602,793915,HKQuantityTypeIdentifierPhysicalEffort,kcal/hr·kg,2024-03-06 09:55:25,2024-03-06 09:50:53,2024-03-06 09:55:01,1.500
128988,115301,HKQuantityTypeIdentifierHeartRate,count/min,2024-03-06 09:58:47,2024-03-06 09:52:51,2024-03-06 09:52:51,52.000
807603,793916,HKQuantityTypeIdentifierPhysicalEffort,kcal/hr·kg,2024-03-06 09:56:26,2024-03-06 09:55:01,2024-03-06 09:55:12,4.800


In [115]:
import datetime as dt

#convert the time/date columns to datetime without time zone
df['creationDate'] = pd.to_datetime(df['creationDate']).dt.tz_localize(None)
df['startDate'] = pd.to_datetime(df['startDate']).dt.tz_localize(None)
df['endDate'] = pd.to_datetime(df['endDate']).dt.tz_localize(None)

EDA Scrubbing - Let's clean the names. To do this, we will split the df into three frames and then reconnect

In [116]:
#create two df, category and non-category
category_df = df[df['type'].str.contains("Category")]
quantity_df = df[df['type'].str.contains("Quantity")]

In [117]:
#scrub the two categories to make the data types more readable
category_df.loc[:,'type'] = category_df['type'].str.replace('HKCategoryTypeIdentifier', "")
quantity_df.loc[:,'type'] = quantity_df['type'].str.replace('HKQuantityTypeIdentifier', "")

In [118]:
#separate into sleep and non-sleep categories
sleep_df = category_df[category_df['type'].str.contains("Sleep")]
non_sleep_df = category_df[~category_df['type'].str.contains("Sleep")]

### Non-Sleep Data - Categorical and non-Categorical

## Scrubbing

In [119]:
#scrub unit column (non_sleep)
non_sleep_df.loc[non_sleep_df['unit'].str.contains("HKCategoryValueAppleStand"),'unit'] = non_sleep_df['unit'].str.replace('HKCategoryValueAppleStand', "")
non_sleep_df.loc[non_sleep_df['unit'].str.contains("HKCategoryValueEnvironmentalAudioExposureEvent"),'unit'] = non_sleep_df['unit'].str.replace("HKCategoryValueEnvironmentalAudioExposureEvent", "")

In [120]:
#scrub quantity df 
quantity_df.loc[:,'type'] = quantity_df['type'] + '_' + quantity_df['unit']
quantity_df = quantity_df.drop('unit', axis = 1)

quantity_df

,Unnamed: 0,type,creationDate,startDate,endDate,value
13687,0,DietaryWater_mL,2023-07-24 10:41:41,2023-07-23 10:41:00,2023-07-23 10:41:00,473.1760
13688,1,DietaryWater_mL,2023-08-23 09:54:44,2023-08-23 09:54:00,2023-08-23 09:54:00,236.5880
13689,2,DietaryWater_mL,2023-08-25 07:02:45,2023-08-25 07:02:00,2023-08-25 07:02:00,473.1760
13690,3,DietaryWater_mL,2023-08-25 07:02:55,2023-08-26 07:02:00,2023-08-26 07:02:00,473.1760
13691,4,DietaryWater_mL,2023-09-05 08:30:09,2023-09-05 08:30:00,2023-09-05 08:30:00,473.1760
...,...,...,...,...,...,...
856980,856980,HeartRateVariabilitySDNN_ms,2024-03-06 01:01:10,2024-03-06 01:00:08,2024-03-06 01:01:07,44.3289
856981,856981,HeartRateVariabilitySDNN_ms,2024-03-06 03:01:11,2024-03-06 03:00:09,2024-03-06 03:01:08,54.3759
856982,856982,HeartRateVariabilitySDNN_ms,2024-03-06 05:01:11,2024-03-06 05:00:09,2024-03-06 05:01:08,76.2300
856983,856983,HeartRateVariabilitySDNN_ms,2024-03-06 07:01:22,2024-03-06 07:00:20,2024-03-06 07:01:20,45.6944


In [121]:
#scrub non-sleep df
non_sleep_df.loc[:,'type'] = non_sleep_df['type'] + '_' + non_sleep_df['unit']
non_sleep_df = non_sleep_df.drop('unit', axis = 1)

non_sleep_df

,Unnamed: 0,type,creationDate,startDate,endDate,value
8144,802061,AppleStandHour_HourStood,2023-07-23 08:10:10,2023-07-23 08:00:00,2023-07-23 09:00:00,1.0
8145,802062,AppleStandHour_HourStood,2023-07-23 09:02:15,2023-07-23 09:00:00,2023-07-23 10:00:00,1.0
8146,802063,AppleStandHour_HourStood,2023-07-23 10:09:32,2023-07-23 10:00:00,2023-07-23 11:00:00,1.0
8147,802064,AppleStandHour_HourStood,2023-07-23 11:16:34,2023-07-23 11:00:00,2023-07-23 12:00:00,1.0
8148,802065,AppleStandHour_HourStood,2023-07-23 12:04:09,2023-07-23 12:00:00,2023-07-23 13:00:00,1.0
...,...,...,...,...,...,...
13682,807599,AudioExposureEvent_MomentaryLimit,2024-01-21 11:51:53,2024-01-21 11:49:03,2024-01-21 11:51:53,1.0
13683,807600,AudioExposureEvent_MomentaryLimit,2024-02-09 10:46:59,2024-02-09 10:44:04,2024-02-09 10:46:59,1.0
13684,807601,AudioExposureEvent_MomentaryLimit,2024-02-12 15:42:33,2024-02-12 15:39:43,2024-02-12 15:42:33,1.0
13685,807602,AudioExposureEvent_MomentaryLimit,2024-02-13 08:30:05,2024-02-13 08:27:10,2024-02-13 08:30:05,1.0


Now that we've done some preliminary scrubbing, let's see if we can create our new dataFrame with only daily information

In [122]:
combined_df = pd.concat([quantity_df, non_sleep_df]).drop(['Unnamed: 0', 'creationDate', 'endDate'], axis = 1)
combined_df = combined_df.sort_values('startDate')

In [123]:
combined_df['startDate'] = pd.to_datetime(combined_df['startDate']).dt.date

In [124]:
combined_df

,type,startDate,value
267737,BasalEnergyBurned_Cal,2023-07-23,76.125
267738,BasalEnergyBurned_Cal,2023-07-23,76.125
267739,BasalEnergyBurned_Cal,2023-07-23,76.125
267740,BasalEnergyBurned_Cal,2023-07-23,76.125
267741,BasalEnergyBurned_Cal,2023-07-23,76.125
...,...,...,...
807601,PhysicalEffort_kcal/hr·kg,2024-03-06,4.800
807602,PhysicalEffort_kcal/hr·kg,2024-03-06,1.500
128988,HeartRate_count/min,2024-03-06,52.000
807603,PhysicalEffort_kcal/hr·kg,2024-03-06,4.800


### Now that we've scrubbed, let's begin creating our new dataframe with daily values

In [16]:
#let's import numpy and datetime
import numpy as np
import datetime as dt

#time/date Initializations
startDate = combined_df.iloc[0,1]
last_index = len(combined_df['startDate']) - 1
lastdate = combined_df.iloc[last_index,1]

#create date range utilizing timedelta function
date_range = np.arange(startDate, lastdate, dt.timedelta(days=1)).astype(dt.date)



In [ ]:
#create new dataframe from data_range and convert date functions back to date time
new_df = pd.DataFrame(date_range, columns = ['date'])
new_df['date'] = pd.to_datetime(new_df['date']).apply(lambda x: x.date())

In [ ]:
new_df = new_df[new_df['date'] > dt.date(2023, 8, 23)]
new_df

In [17]:
#let's create our new dataframe with the appropriate columns that we wish to see. For each potential column name, we'll decide what daily values we want to see

for col in combined_df['type'].unique():
    if "/" not in col:
        col_names = col   
    elif "/" in col:
        col_names = [col+'_mean',col+'_min', col+'_max']   
    new_df[col_names]=np.nan

new_df

,date,BasalEnergyBurned_Cal,DietaryCholesterol_mg,DietaryMagnesium_mg,DietaryIodine_mcg,DietarySodium_mg,DietaryPhosphorus_mg,DietaryZinc_mg,DietaryCarbohydrates_g,DietaryFiber_g,...,RunningVerticalOscillation_cm,RunningGroundContactTime_ms,BodyMass_lb,DietaryCaffeine_mg,DistanceSwimming_yd,SwimmingStrokeCount_count,DietaryMolybdenum_mcg,PhysicalEffort_kcal/hr·kg_mean,PhysicalEffort_kcal/hr·kg_min,PhysicalEffort_kcal/hr·kg_max
0,2023-07-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-07-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-07-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-07-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-07-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222,2024-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
223,2024-03-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
224,2024-03-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
225,2024-03-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
day = startDate
for col in combined_df['type'].unique():
    for day in new_df['date']:
        export = combined_df[(combined_df['type'] == col) & (combined_df['startDate'] == day)]['value']
        date_index = new_df.loc[new_df['date'] == day].index[0]
        if "/" not in col:
            new_df.loc[date_index, col] = export.sum()
        elif "/" in col:
            new_df.loc[date_index, [col+'_mean',col+'_min', col+'_max']] = [export.mean(), export.min(), export.max()]


In [51]:
new_df

,date,BasalEnergyBurned_Cal,DietaryCholesterol_mg,DietaryMagnesium_mg,DietaryIodine_mcg,DietarySodium_mg,DietaryPhosphorus_mg,DietaryZinc_mg,DietaryCarbohydrates_g,DietaryFiber_g,...,RunningVerticalOscillation_cm,RunningGroundContactTime_ms,BodyMass_lb,DietaryCaffeine_mg,DistanceSwimming_yd,SwimmingStrokeCount_count,DietaryMolybdenum_mcg,PhysicalEffort_kcal/hr·kg_mean,PhysicalEffort_kcal/hr·kg_min,PhysicalEffort_kcal/hr·kg_max
32,2023-08-24,2055.322,1131.38,138.0,0.0,17596.94,366.0,1.0,810.8826,103.5022,...,0.00000,0.000,196.9,0.0,0.0,0.0,0.0,NaN,NaN,NaN
33,2023-08-25,2174.950,593.20,0.0,0.0,7721.20,0.0,0.0,305.5500,38.3640,...,0.00000,0.000,195.1,0.0,0.0,0.0,0.0,NaN,NaN,NaN
34,2023-08-26,2074.476,760.00,182.0,0.0,9170.00,640.0,2.8,439.0000,29.6000,...,0.00000,0.000,195.1,20.0,0.0,0.0,0.0,NaN,NaN,NaN
35,2023-08-27,2187.383,780.00,150.0,0.0,12238.00,486.0,3.0,342.6800,88.0400,...,20.41179,598.327,192.9,0.0,0.0,0.0,0.0,NaN,NaN,NaN
36,2023-08-28,2186.244,750.00,110.0,0.0,11182.00,396.0,2.6,420.8000,52.8000,...,0.00000,0.000,192.9,12.0,0.0,0.0,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222,2024-03-01,2004.932,304.00,370.0,0.0,8462.00,1500.0,10.2,638.0000,71.0000,...,0.00000,0.000,174.4,0.0,0.0,0.0,0.0,NaN,NaN,NaN
223,2024-03-02,2048.925,364.00,242.0,0.0,9770.00,496.0,2.2,451.4000,47.2000,...,0.00000,0.000,174.6,0.0,0.0,0.0,0.0,NaN,NaN,NaN
224,2024-03-03,2048.189,578.00,276.0,0.0,6112.00,744.0,6.4,478.6000,39.0000,...,0.00000,0.000,175.0,0.0,525.0,378.0,0.0,4.044340,1.0,10.9
225,2024-03-04,1983.933,586.00,208.0,0.0,5632.00,604.0,5.6,605.6000,43.2000,...,0.00000,0.000,175.7,0.0,0.0,0.0,0.0,3.347815,1.0,12.9


In [52]:
new_df = new_df[new_df['date'] > dt.date(2023, 8, 23)]
new_df

,date,BasalEnergyBurned_Cal,DietaryCholesterol_mg,DietaryMagnesium_mg,DietaryIodine_mcg,DietarySodium_mg,DietaryPhosphorus_mg,DietaryZinc_mg,DietaryCarbohydrates_g,DietaryFiber_g,...,RunningVerticalOscillation_cm,RunningGroundContactTime_ms,BodyMass_lb,DietaryCaffeine_mg,DistanceSwimming_yd,SwimmingStrokeCount_count,DietaryMolybdenum_mcg,PhysicalEffort_kcal/hr·kg_mean,PhysicalEffort_kcal/hr·kg_min,PhysicalEffort_kcal/hr·kg_max
32,2023-08-24,2055.322,1131.38,138.0,0.0,17596.94,366.0,1.0,810.8826,103.5022,...,0.00000,0.000,196.9,0.0,0.0,0.0,0.0,NaN,NaN,NaN
33,2023-08-25,2174.950,593.20,0.0,0.0,7721.20,0.0,0.0,305.5500,38.3640,...,0.00000,0.000,195.1,0.0,0.0,0.0,0.0,NaN,NaN,NaN
34,2023-08-26,2074.476,760.00,182.0,0.0,9170.00,640.0,2.8,439.0000,29.6000,...,0.00000,0.000,195.1,20.0,0.0,0.0,0.0,NaN,NaN,NaN
35,2023-08-27,2187.383,780.00,150.0,0.0,12238.00,486.0,3.0,342.6800,88.0400,...,20.41179,598.327,192.9,0.0,0.0,0.0,0.0,NaN,NaN,NaN
36,2023-08-28,2186.244,750.00,110.0,0.0,11182.00,396.0,2.6,420.8000,52.8000,...,0.00000,0.000,192.9,12.0,0.0,0.0,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222,2024-03-01,2004.932,304.00,370.0,0.0,8462.00,1500.0,10.2,638.0000,71.0000,...,0.00000,0.000,174.4,0.0,0.0,0.0,0.0,NaN,NaN,NaN
223,2024-03-02,2048.925,364.00,242.0,0.0,9770.00,496.0,2.2,451.4000,47.2000,...,0.00000,0.000,174.6,0.0,0.0,0.0,0.0,NaN,NaN,NaN
224,2024-03-03,2048.189,578.00,276.0,0.0,6112.00,744.0,6.4,478.6000,39.0000,...,0.00000,0.000,175.0,0.0,525.0,378.0,0.0,4.044340,1.0,10.9
225,2024-03-04,1983.933,586.00,208.0,0.0,5632.00,604.0,5.6,605.6000,43.2000,...,0.00000,0.000,175.7,0.0,0.0,0.0,0.0,3.347815,1.0,12.9


### Sleep scrubbing

In [125]:
#scrub unit column (sleep)
sleep_df.loc[sleep_df['unit'].str.contains("HKCategoryValueSleepAnalysis"),'unit'] = sleep_df['unit'].str.replace("HKCategoryValueSleepAnalysis", "")

In [126]:
#scrub sleep column
sleep_df.loc[:,'type'] = sleep_df['type'] + '_' + sleep_df['unit'] + '_hrs'
sleep_df = sleep_df.drop('unit', axis = 1)

In [127]:
#sleep_df['startDate'] = pd.to_datetime(sleep_df['startDate']).dt.date
sleep_df['startDate'] = pd.to_datetime(sleep_df['startDate'], format='%d%b%Y:%H:%M:%S')
sleep_df['endDate'] = pd.to_datetime(sleep_df['endDate'], format='%d%b%Y:%H:%M:%S')
sleep_df['value'] = (sleep_df['endDate'] - sleep_df['startDate'])/dt.timedelta(hours=1)

#remove unwanted sleep columns
sleep_df = sleep_df[sleep_df['type'] != 'SleepAnalysis_InBed_hrs']
sleep_df = sleep_df[sleep_df['type'] != 'SleepAnalysis_AsleepUnspecified_hrs']

In [128]:
sleep_df

,Unnamed: 0,type,creationDate,startDate,endDate,value
3,793920,SleepAnalysis_AsleepCore_hrs,2023-07-24 07:05:22,2023-07-23 23:12:48,2023-07-23 23:40:48,0.466667
4,793921,SleepAnalysis_AsleepDeep_hrs,2023-07-24 07:05:22,2023-07-23 23:40:48,2023-07-24 00:06:18,0.425000
5,793922,SleepAnalysis_AsleepCore_hrs,2023-07-24 07:05:22,2023-07-24 00:06:18,2023-07-24 00:12:18,0.100000
6,793923,SleepAnalysis_AsleepREM_hrs,2023-07-24 07:05:22,2023-07-24 00:12:18,2023-07-24 00:27:48,0.258333
7,793924,SleepAnalysis_AsleepCore_hrs,2023-07-24 07:05:22,2023-07-24 00:27:48,2023-07-24 00:31:48,0.066667
...,...,...,...,...,...,...
8130,802047,SleepAnalysis_AsleepCore_hrs,2024-03-06 07:12:34,2024-03-06 04:56:55,2024-03-06 05:43:25,0.775000
8131,802048,SleepAnalysis_AsleepREM_hrs,2024-03-06 07:12:34,2024-03-06 05:43:25,2024-03-06 05:56:55,0.225000
8132,802049,SleepAnalysis_AsleepCore_hrs,2024-03-06 07:12:34,2024-03-06 05:56:55,2024-03-06 05:59:25,0.041667
8133,802050,SleepAnalysis_AsleepREM_hrs,2024-03-06 07:12:34,2024-03-06 05:59:25,2024-03-06 06:21:55,0.375000


In [129]:
#sleep_df = sleep_df.drop(['Unnamed: 0', 'creationDate', 'endDate'], axis = 1)
def sleep_date (date):
    if date.hour > 11:
        return date
    else:
        return (date - pd.Timedelta(1, unit='D'))

sleep_df['date'] = sleep_df['startDate'].apply(sleep_date)
sleep_df['date'] = pd.to_datetime(sleep_df['date']).dt.date 
sleep_df 


,Unnamed: 0,type,creationDate,startDate,endDate,value,date
3,793920,SleepAnalysis_AsleepCore_hrs,2023-07-24 07:05:22,2023-07-23 23:12:48,2023-07-23 23:40:48,0.466667,2023-07-23
4,793921,SleepAnalysis_AsleepDeep_hrs,2023-07-24 07:05:22,2023-07-23 23:40:48,2023-07-24 00:06:18,0.425000,2023-07-23
5,793922,SleepAnalysis_AsleepCore_hrs,2023-07-24 07:05:22,2023-07-24 00:06:18,2023-07-24 00:12:18,0.100000,2023-07-23
6,793923,SleepAnalysis_AsleepREM_hrs,2023-07-24 07:05:22,2023-07-24 00:12:18,2023-07-24 00:27:48,0.258333,2023-07-23
7,793924,SleepAnalysis_AsleepCore_hrs,2023-07-24 07:05:22,2023-07-24 00:27:48,2023-07-24 00:31:48,0.066667,2023-07-23
...,...,...,...,...,...,...,...
8130,802047,SleepAnalysis_AsleepCore_hrs,2024-03-06 07:12:34,2024-03-06 04:56:55,2024-03-06 05:43:25,0.775000,2024-03-05
8131,802048,SleepAnalysis_AsleepREM_hrs,2024-03-06 07:12:34,2024-03-06 05:43:25,2024-03-06 05:56:55,0.225000,2024-03-05
8132,802049,SleepAnalysis_AsleepCore_hrs,2024-03-06 07:12:34,2024-03-06 05:56:55,2024-03-06 05:59:25,0.041667,2024-03-05
8133,802050,SleepAnalysis_AsleepREM_hrs,2024-03-06 07:12:34,2024-03-06 05:59:25,2024-03-06 06:21:55,0.375000,2024-03-05


In [132]:
#let's create our new dataframe with the appropriate columns that we wish to see. For each potential column name, we'll decide what daily values we want to see
new_sleep = pd.DataFrame(date_range, columns = ['date'])
new_sleep['date'] = pd.to_datetime(new_df['date']).apply(lambda x: x.date())
new_sleep = new_sleep[new_sleep['date'] > dt.date(2023, 8, 23)]
new_sleep

,date
32,2023-08-24
33,2023-08-25
34,2023-08-26
35,2023-08-27
36,2023-08-28
...,...
222,2024-03-01
223,2024-03-02
224,2024-03-03
225,2024-03-04


In [133]:
for col in sleep_df['type'].unique():
    for day in new_sleep['date']:
        export = sleep_df[(sleep_df['type'] == col) & (sleep_df['date'] == day)]['value']
        date_index = new_sleep.loc[new_sleep['date'] == day].index[0]
        new_sleep.loc[date_index, col] = export.sum()

In [134]:
new_sleep

,date,SleepAnalysis_AsleepCore_hrs,SleepAnalysis_AsleepDeep_hrs,SleepAnalysis_AsleepREM_hrs,SleepAnalysis_Awake_hrs
32,2023-08-24,5.558333,0.783333,1.766667,0.266667
33,2023-08-25,3.700000,1.008333,1.500000,0.133333
34,2023-08-26,3.916667,1.400000,1.558333,0.050000
35,2023-08-27,5.566667,0.891667,2.591667,0.066667
36,2023-08-28,5.275000,0.641667,2.008333,0.158333
...,...,...,...,...,...
222,2024-03-01,4.475000,0.700000,1.683333,2.525000
223,2024-03-02,0.000000,0.000000,0.000000,0.000000
224,2024-03-03,3.925000,1.108333,1.966667,0.300000
225,2024-03-04,0.000000,0.000000,0.000000,0.000000


### Merge Data

In [135]:
merged_df = pd.merge(new_df, new_sleep, on='date', how='inner')

In [136]:
merged_df

,date,BasalEnergyBurned_Cal,DietaryCholesterol_mg,DietaryMagnesium_mg,DietaryIodine_mcg,DietarySodium_mg,DietaryPhosphorus_mg,DietaryZinc_mg,DietaryCarbohydrates_g,DietaryFiber_g,...,DistanceSwimming_yd,SwimmingStrokeCount_count,DietaryMolybdenum_mcg,PhysicalEffort_kcal/hr·kg_mean,PhysicalEffort_kcal/hr·kg_min,PhysicalEffort_kcal/hr·kg_max,SleepAnalysis_AsleepCore_hrs,SleepAnalysis_AsleepDeep_hrs,SleepAnalysis_AsleepREM_hrs,SleepAnalysis_Awake_hrs
0,2023-08-24,2055.322,1131.38,138.0,0.0,17596.94,366.0,1.0,810.8826,103.5022,...,0.0,0.0,0.0,NaN,NaN,NaN,5.558333,0.783333,1.766667,0.266667
1,2023-08-25,2174.950,593.20,0.0,0.0,7721.20,0.0,0.0,305.5500,38.3640,...,0.0,0.0,0.0,NaN,NaN,NaN,3.700000,1.008333,1.500000,0.133333
2,2023-08-26,2074.476,760.00,182.0,0.0,9170.00,640.0,2.8,439.0000,29.6000,...,0.0,0.0,0.0,NaN,NaN,NaN,3.916667,1.400000,1.558333,0.050000
3,2023-08-27,2187.383,780.00,150.0,0.0,12238.00,486.0,3.0,342.6800,88.0400,...,0.0,0.0,0.0,NaN,NaN,NaN,5.566667,0.891667,2.591667,0.066667
4,2023-08-28,2186.244,750.00,110.0,0.0,11182.00,396.0,2.6,420.8000,52.8000,...,0.0,0.0,0.0,NaN,NaN,NaN,5.275000,0.641667,2.008333,0.158333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,2024-03-01,2004.932,304.00,370.0,0.0,8462.00,1500.0,10.2,638.0000,71.0000,...,0.0,0.0,0.0,NaN,NaN,NaN,4.475000,0.700000,1.683333,2.525000
191,2024-03-02,2048.925,364.00,242.0,0.0,9770.00,496.0,2.2,451.4000,47.2000,...,0.0,0.0,0.0,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000
192,2024-03-03,2048.189,578.00,276.0,0.0,6112.00,744.0,6.4,478.6000,39.0000,...,525.0,378.0,0.0,4.044340,1.0,10.9,3.925000,1.108333,1.966667,0.300000
193,2024-03-04,1983.933,586.00,208.0,0.0,5632.00,604.0,5.6,605.6000,43.2000,...,0.0,0.0,0.0,3.347815,1.0,12.9,0.000000,0.000000,0.000000,0.000000


In [139]:
filepath = 'pre_kaggle/merged_health.csv'

# Export the DataFrame to the specified file
merged_df.to_csv(filepath)